# Getting access to Mat's Data

In [1]:
#This gets us into the right directory from home, in order to run the import python script from Mat

#sys = system (module), gives information and control over python interpreter/terminal itself
import sys
sys.path.append("/home/565/pv3484/aus_substation_electricity-main")

#% is a magic command, special shortcut command that lets you control/interact with notebook environment (ie. gives control of terminal without writing full python code)
%cd aus_substation_electricity-main/

!pwd

/home/565/pv3484/aus_substation_electricity-main
/home/565/pv3484/aus_substation_electricity-main


In [2]:
#This section imports the substations that Mat put together

%run /home/565/pv3484/aus_substation_electricity-main/import_substation.py

processing nsw substations for ['ausgrid'] from None to None
ausgrid
following columns in demand are not in info index:
['MT_HU', 'SI_NO']
removing these columns from demand
number of substations in ausgrid substation info: 134
number of substations in ausgrid substation data: 132
following sites match selection criteria:
               energy_asset          Name  Area  Dwellings  Persons  Residential  Commercial  Industrial  Primary Production  Education  \
ID                                                                                                                                        
BLAKE         AG_BLAKEHURST    Blakehurst     7      10081    28521        0.850       0.005       0.021               0.000      0.022   
PUNCH          AG_PUNCHBOWL     Punchbowl     9      17514    50395        0.826       0.048       0.038               0.000      0.025   
MEADO         AG_MEADOWBANK    Meadowbank    15      22420    56948        0.825       0.023       0.015               0

In [9]:
#print names in each column of the info variable

print("Column names in info:",info.columns)

#residential column 6

Column names in info: Index(['energy_asset', 'Name', 'Area', 'Dwellings', 'Persons', 'Residential', 'Commercial', 'Industrial', 'Primary Production', 'Education',
       'Hospital/Medical', 'Transport', 'Parkland', 'Water', 'Other'],
      dtype='object')


# Trying geocode (geopy)

## list of substation names

In [21]:
#print list of substation names

residential_names = info[info['Residential'] > 0.5]['Name']
print(residential_names)

BLAKE         Blakehurst
PUNCH          Punchbowl
MEADO         Meadowbank
MOSMA             Mosman
RIVER          Riverwood
CA_CO        Castle Cove
DU_HI       Dulwich Hill
FI_DO          Five Dock
UMINA              Umina
CAMPS            Campsie
HU_HI       Hunters Hill
MORTD           Mortdale
CHATS          Chatswood
KOGAR            Kogarah
PE_HI      Pennant Hills
LIDCO           Lidcombe
BURWO            Burwood
HORNS            Hornsby
EPPIN             Epping
DE_WH       Dee Why West
HARBO            Harbord
SEFTO             Sefton
MARRI       Marrickville
GR_PA     Greenacre Park
ENFIE            Enfield
ARNCL          Arncliffe
REVES            Revesby
CARDI            Cardiff
TO_RY           Top Ryde
NARRA          Narrabeen
BANKS          Bankstown
CONCO            Concord
CARIN          Caringbah
BA_HI          Bass Hill
NE_CB      Newcastle Cbd
MAROU           Maroubra
BA_NO    Balgowlah North
ADAMS          Adamstown
JESMO            Jesmond
KIRRA           Kirrawee


# Trialing different mapping

## folium interative map

### initial mapping

In [44]:
#seeing what's in the 'info' variable

info.head(10)

,energy_asset,Name,Area,Dwellings,Persons,Residential,Commercial,Industrial,Primary Production,Education,Hospital/Medical,Transport,Parkland,Water,Other,Latitude,Longitude
BLAKE,AG_BLAKEHURST,Blakehurst,7,10081,28521,0.850,0.005,0.021,0.0,0.022,0.001,0.000,0.102,0.000,0.0,-33.989852,151.108560
PUNCH,AG_PUNCHBOWL,Punchbowl,9,17514,50395,0.826,0.048,0.038,0.0,0.025,0.002,0.001,0.061,0.000,0.0,-33.928717,151.052259
MEADO,AG_MEADOWBANK,Meadowbank,15,22420,56948,0.825,0.023,0.015,0.0,0.036,0.014,0.006,0.082,0.000,0.0,-33.817499,151.088425
MOSMA,AG_MOSMAN,Mosman,11,25967,52831,0.817,0.046,0.001,0.0,0.013,0.001,0.000,0.123,0.000,0.0,-33.830135,151.244766
RIVER,AG_RIVERWOOD,Riverwood,10,12528,34547,0.813,0.018,0.042,0.0,0.035,0.002,0.001,0.089,0.000,0.0,-33.947222,151.053056
CA_CO,AG_CASTLE COVE,Castle Cove,10,11682,30714,0.807,0.033,0.031,0.0,0.017,0.000,0.002,0.110,0.001,0.0,-33.786529,151.214319
DU_HI,AG_DULWICH HILL,Dulwich Hill,9,18323,43563,0.799,0.023,0.007,0.0,0.036,0.007,0.005,0.122,0.000,0.0,-33.904585,151.138609
FI_DO,AG_FIVE DOCK,Five Dock,8,16012,38795,0.798,0.044,0.020,0.0,0.032,0.004,0.007,0.095,0.000,0.0,-33.869002,151.129023
UMINA,AG_UMINA,Umina,16,13382,26489,0.790,0.013,0.001,0.0,0.022,0.000,0.000,0.174,0.000,0.0,-33.517778,151.309444
CAMPS,AG_CAMPSIE,Campsie,16,25453,68810,0.789,0.025,0.057,0.0,0.016,0.003,0.004,0.107,0.000,0.0,-33.914389,151.103213


### fixing locations with missing lat or lan 

In [55]:
# Filter rows where Latitude or Longitude is missing
missing_coords = info[info['Latitude'].isnull() | info['Longitude'].isnull()]

# Print all columns for those rows
print(missing_coords[['Name', 'Residential', 'Latitude', 'Longitude']])


               Name  Residential  Latitude  Longitude
DE_WH  Dee Why West        0.722       NaN        NaN


In [56]:
#adding coordinates for Dee Why West location

#lat: -33.7511
#lon: 151.2889

#update dataframe
info.loc[info['Name'] == 'Dee Why West', ['Latitude', 'Longitude']] = [-33.7511, 151.2889]

#verify update
print(info.loc[info['Name'] == 'Dee Why West', ['Name', 'Latitude', 'Longitude']])

               Name  Latitude  Longitude
DE_WH  Dee Why West  -33.7511   151.2889


In [58]:
#create new map with all lon lat locations filled

import folium

# Use the full DataFrame now that lat/lon are filled
m = folium.Map(
    location=[info['Latitude'].mean(), info['Longitude'].mean()],
    zoom_start=10
)

# Add markers for every substation
for _, row in info.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name']
    ).add_to(m)

m

### colour code map by residential fraction

In [85]:
import folium

m = folium.Map(
    location=[info['Latitude'].mean(), info['Longitude'].mean()],
    zoom_start=10
)

# Add color-coded markers
for _, row in info.iterrows():
    if row['Residential'] > 0.5:
        color = 'green'
    elif row['Residential'] > 0.2:
        color = 'orange'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['Name']} (Residential fraction: {row['Residential']:.2f})"
    ).add_to(m)

# Add legend with HTML
legend_html = """
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8;
     ">
&nbsp;<b>Residential Fraction</b><br>
&nbsp;<i style="color:green;">●</i> > 0.5<br>
&nbsp;<i style="color:orange;">●</i> 0.2 – 0.5<br>
&nbsp;<i style="color:red;">●</i> < 0.2
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m


In [90]:
m.fit_bounds(m.get_bounds())   # ensures all substations are visible
m.save("substations_map.html")